In [4]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms


In [5]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [6]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 40, 40])

In [7]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

In [11]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)

In [12]:
device = torch.device('cuda')
model = model.to(device)

In [13]:
import datetime
today = datetime.date.today()
strToday = str(today.year) + '_' + str(today.month) + '_' + str(today.day)

RESNET_MODEL = 'Botline_CA_model_resnet18_' + strToday + '.pth'
TRT_MODEL = 'Botline_CA_model_resnet18_trt_' + strToday + '.pth'

In [64]:
NUM_EPOCHS = 20
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), RESNET_MODEL)
        best_accuracy = test_accuracy

0: 0.950000
1: 0.975000
2: 0.950000
3: 0.975000
4: 1.000000
5: 0.975000
6: 1.000000
7: 0.975000
8: 0.950000
9: 0.925000
10: 0.975000
11: 0.925000
12: 0.975000
13: 0.925000
14: 0.950000
15: 1.000000
16: 1.000000
17: 1.000000
18: 0.950000
19: 0.975000


In [14]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

model.load_state_dict(torch.load(RESNET_MODEL))

<All keys matched successfully>

In [9]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
torch.save(model_trt.state_dict(), TRT_MODEL)